# Train and Apply Models

In [1]:
from ML.model_training import (
    train_lstm,
)
from ML.labels import build_video_rating_tables
from ML.splits import single_user_split
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
)
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd
import math, re, itertools
from ML import utils
import sys
from IPython.display import clear_output
from scipy.stats import pearsonr

Generate all subsets of columns for parameters.

In [3]:
X_train, X_test, arousal_train, arousal_test = single_user_split(
    target="arousal", selected_user=0, k_holdouts=1
)
features = utils.filter_features(X_train.columns, remove_bands=["gamma", "delta"])
arousal_train = pd.Series(
    np.where(arousal_train > 3.8, "high", "low"),
    index=arousal_train.index,
    dtype="string",
)
arousal_test = pd.Series(
    np.where(arousal_test > 3.8, "high", "low"),
    index=arousal_test.index,
    dtype="string",
)


def balance(X, y, seed=5):
    c = y.value_counts()
    if c.get("high", 0) == c.get("low", 0):
        return X.reset_index(drop=True), y.reset_index(drop=True)
    maj = c.idxmax()
    m = c.min()
    keep = y[y != maj].index.union(y[y == maj].sample(m, random_state=seed).index)
    return X.loc[keep].reset_index(drop=True), y.loc[keep].reset_index(drop=True)


X_train, arousal_train = balance(X_train, arousal_train, seed=5)
X_test, arousal_test = balance(X_test, arousal_test, seed=5)

print("arousal_train counts:\n", arousal_train.value_counts(dropna=False))
print("arousal_test counts:\n", arousal_test.value_counts(dropna=False))

arousal_train counts:
 low     638
high    638
Name: count, dtype: Int64
arousal_test counts:
 high    196
Name: count, dtype: Int64


## LSTM LOO

In [5]:
best_model = None
best_acc = 0
best_keep = None

best_lr = 0
best_f1 = 0
bar_len = 30

status = f"Best: index= size= | " f"acc= | f1= | prec= | rec="

results = []
X_train_sub = X_train.loc[:, features]
X_test_sub = X_test.loc[:, features]

n_low = (arousal_train == "low").sum()
n_high = (arousal_train == "high").sum()
for lr in [0.0001]:
    for e in [10]:
        for u in [128]:
            for b_s in [128]:
                lstm, X_test_eval, y_test_eval = train_lstm(
                    X_train_sub,
                    X_test_sub,
                    arousal_train,
                    arousal_test,
                    lr=lr,
                    epochs=e,
                    units=u,
                    batch_size=b_s,
                    bidirectional=False,
                )
                y_prob = lstm.predict(X_test_eval).ravel()
                arousal_pred = (y_prob >= 0.5).astype(int)

                acc = accuracy_score(y_test_eval, arousal_pred)
                f1 = f1_score(y_test_eval, arousal_pred, average="weighted")
                prec = precision_score(y_test_eval, arousal_pred, average="weighted")
                rec = recall_score(y_test_eval, arousal_pred, average="weighted")

                if acc > best_acc:
                    best_acc = acc
                    best_model = lstm
                    best_lr = lr
                    best_e = e
                    best_u = u
                    best_b_s = b_s
                    best_f1 = f1
                    best_arousal_pred = arousal_pred
                    status = (
                        f"Best: "
                        f"acc={acc:.6f} | f1={f1:.6f} | prec={prec:.6f} | rec={rec:.6f} | lr={best_lr} | epochs={best_e} | units={best_u} | batch_size={best_b_s}"
                    )
                    print(status)

print("\nConfusion Matrix (pooled):")
print(confusion_matrix(y_test_eval, best_arousal_pred))

print("\nClassification Report (pooled):")
print(classification_report(y_test_eval, best_arousal_pred, zero_division=0))

print(status)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Best: acc=0.020408 | f1=0.040000 | prec=1.000000 | rec=0.020408 | lr=0.0001 | epochs=10 | units=128 | batch_size=128

Confusion Matrix (pooled):
[[  0   0]
 [192   4]]

Classification Report (pooled):
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.02      0.04       196

    accuracy                           0.02       196
   macro avg       0.50      0.01      0.02       196
weighted avg       1.00      0.02      0.04       196

Best: acc=0.020408 | f1=0.040000 | prec=1.000000 | rec=0.020408 | lr=0.0001 | epochs=10 | units=128 | batch_size=128


/Users/tolas/Documents/coding/Tetris-EEG/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


## Choose participants for training

In [6]:
# define the subjects you want to use as "folds"
subjects = [i for i in range(0, 23)]
videos = [j for j in range(0, 18)]


# hyper params
lr_grid = [0.0001]
epochs_grid = [10]
units_grid = [128]
batch_size_grid = [128]

best_lr = None
best_e = None
best_u = None
best_b_s = None
best_cv_acc = -np.inf
total_size = 0

for lr in lr_grid:
    for e in epochs_grid:
        for u in units_grid:
            for b_s in batch_size_grid:
                y_true_all = []
                y_pred_all = []
                acc_list = []

                for i in subjects:
                    for j in videos:
                        while True:
                            X_train, X_test, arousal_train, arousal_test = (
                                single_user_split(
                                    target="arousal",
                                    k_holdouts=1,
                                    selected_user=i,
                                    holdout_videos=[j],
                                )
                            )
                            features = utils.filter_features(
                                X_train.columns, remove_bands=["gamma", "delta"]
                            )
                            X_train = X_train.loc[:, features]
                            X_test = X_test.loc[:, features]
                            arousal_train = pd.Series(
                                np.where(arousal_train > 3.8, "high", "low"),
                                index=arousal_train.index,
                                dtype="string",
                            )
                            arousal_test = pd.Series(
                                np.where(arousal_test > 3.8, "high", "low"),
                                index=arousal_test.index,
                                dtype="string",
                            )

                            # if you want to enforce balance conditions, re-enable these:
                            # c = arousal_test.value_counts()
                            # if c.get("high", 0) == 0 or c.get("low", 0) == 0:
                            #     continue
                            # if c.get("low", 0) * 5 >= arousal_train.value_counts().get("low", 0):
                            #     continue

                            # if you want balancing:
                            # X_train, arousal_train = balance(X_train, arousal_train)
                            # X_test, arousal_test = balance(X_test, arousal_test)

                            break

                        lstm, X_test_eval, y_test_eval = train_lstm(
                            X_train,
                            X_test,
                            arousal_train,
                            arousal_test,
                            lr=lr,
                            epochs=e,
                            units=u,
                            batch_size=b_s,
                            bidirectional=False,
                        )

                        y_prob = lstm.predict(X_test_eval, verbose=0).ravel()
                        arousal_pred = (y_prob >= 0.5).astype(int)

                        acc = accuracy_score(y_test_eval, arousal_pred)
                        acc_list.append(float(acc))

                        y_true_all.extend(y_test_eval.tolist())
                        y_pred_all.extend(arousal_pred.tolist())

                    print(
                        np.mean(acc_list),
                        np.mean(acc_list[len(acc_list) - 18 : len(acc_list)]),
                    )

                mean_acc = float(np.mean(acc_list))
                cm = confusion_matrix(y_true_all, y_pred_all, labels=[0, 1])
                overall_acc = accuracy_score(y_true_all, y_pred_all)

                print(
                    f"lr={lr}, epochs={e}, units={u}, batch_size={b_s} "
                    f"-> mean CV acc={mean_acc:.4f}, pooled overall acc={overall_acc:.4f}"
                )
                print(
                    "Pooled confusion matrix for this setting (rows=true, cols=pred):"
                )
                print(cm)

                # update best based on mean CV accuracy (your current selection criterion)
                if mean_acc > best_cv_acc:
                    best_cv_acc = mean_acc
                    best_lr = lr
                    best_e = e
                    best_u = u
                    best_b_s = b_s
                    best_cm = cm
                    best_overall_acc = overall_acc

print("\nBest hyperparameters (cross-subject CV):")
print(f"  lr={best_lr}")
print(f"  epochs={best_e}")
print(f"  units={best_u}")
print(f"  batch_size={best_b_s}")
print(f"  mean CV accuracy (unweighted over folds) = {best_cv_acc:.4f}")

print("\nPooled metrics for best hyperparameters:")
print("  Confusion matrix (rows=true, cols=pred):")
print(best_cm)
print(f"  Overall pooled accuracy = {best_overall_acc:.4f}")

KeyboardInterrupt: 